In [48]:
import pandas as pd
import numpy as np

game_session = pd.read_csv("../Dataset 2.0/Game Sessions.csv")
uniquePlayers = np.unique(game_session["User ID"])

users = []
for i in uniquePlayers:
    tmp = game_session[game_session["User ID"] == i]
    if len(tmp) > 30:
        users.append(i)
        
print(f"Sono presenti {len(users)} users con più di 30 game session giocate: {users}")

Sono presenti 34 users con più di 30 game session giocate: ['000a62d8-5e6d-4532-ad99-af8a92a7e7f3', '00212664-6c71-4e24-b61d-c7776f7a7d99', '12b5a3b6-ccd8-4259-9943-ea4d34a89a1a', '28c1a568-4a34-4c0a-90c9-da8150390e88', '2fe37a4e-2f03-4573-844f-ee523105dc00', '3452d36a-8ed4-4253-b9db-d8fbeb74638e', '453c69f9-3e08-46f4-9c34-b036575a342e', '46da2a87-6999-4690-a45f-89f69c9d9a15', '4a7b26aa-bed3-4660-8014-fa2d51e21030', '4a8258fd-a3fc-4816-afc7-9f62a17a0664', '574e89d2-8320-4f63-a8e4-4d25fb9b3e8d', '6afc891c-1442-4d1d-9660-00e28fa16489', '6e9c88f4-0b93-4c2f-a681-a72a30e41b64', '704e4795-fc54-4e87-a764-aa49ba52011d', '70506f5d-00ac-4ac5-9e9c-6fd6406016af', '74ddab53-3849-4848-a0df-efb50e0eba8f', '85c24634-825e-4f55-99e7-fe3c1c00ebbc', '863b4b11-6da2-44b9-8e21-4f5954ed5b04', '8db00f0c-87e8-4515-b73e-2e67244c7198', '92927f86-f921-4e6d-996e-9f337b6b2877', '9caf068d-5e86-4818-8d83-379ea55d4e6f', '9da6520c-97c9-4c23-9d6a-ea5a603b5163', 'adc3fcd1-e4bf-42a5-b9c6-5a9a0b307cef', 'bdf52837-f61c-402a-

In [49]:
lista = []
for i in users:
    tmp = game_session[game_session["User ID"] == i]
    tmp = tmp[tmp["Game Level"] == 5]
    for i in range(len(tmp)):
        lista.append(tmp.iloc[i, 0])
        
lista = np.array(lista)
lista = np.unique(lista)

In [50]:
#Importo le informazioni riguardanti le qualità delle tasks 
tasks = pd.read_csv("../Dataset 2.0/Tasks.csv")
tasks = tasks.drop(0, axis = 0)
tasks = tasks.drop("Deadline", axis = 1)


#Importo i coefficienti di regressione di ogni agente
coeffs = pd.read_csv("../Coefficienti di Regressione/coeffs.csv")

decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")
decisions = decisions[decisions["Round"] == 9]
decisions

,ID,Session ID,Round,Worker Agent ID,Worker Agent Reputation
630,1521,cd6b97eb-16f5-486e-a29b-c3baf4c2baec,9,11,0.093912
631,1523,cd6b97eb-16f5-486e-a29b-c3baf4c2baec,9,16,0.307397
632,1525,cd6b97eb-16f5-486e-a29b-c3baf4c2baec,9,20,0.319685
633,1527,cd6b97eb-16f5-486e-a29b-c3baf4c2baec,9,13,0.160180
634,1529,cd6b97eb-16f5-486e-a29b-c3baf4c2baec,9,14,0.313495
...,...,...,...,...,...
495388,1290646,b206c2c6-de43-414a-abbf-af57e750f906,9,16,0.548162
495389,1290648,b206c2c6-de43-414a-abbf-af57e750f906,9,17,0.690501
495390,1290650,b206c2c6-de43-414a-abbf-af57e750f906,9,18,0.577292
495391,1290652,b206c2c6-de43-414a-abbf-af57e750f906,9,19,0.774562


In [51]:
from mip import *


for i in lista:
    
    print("\n")
    print(i)
    #Selezionare il Session ID ed il Round voluti
    #Assicurarsi di selezionare lo stesso round in "RegressionModel.py" e in "OptimizationModel.py"
    decisions1 = decisions[decisions["Session ID"] == i]
    decisions1 = decisions1.sort_values("Worker Agent ID")
    m = Model()

    #DATI

    #Identifier del Round
    dID = list(decisions1["ID"])


    #Peso dell'Identifier del round
    wdID = coeffs.iloc[0].values.tolist()


    #Identifier del Worker Agent
    WAID = list(decisions1["Worker Agent ID"])


    #Peso dell'Identifier del WA
    wWAID = coeffs.iloc[91].values.tolist()

    if (WAID[9] == 10):
        wWAID = wWAID[0:10]

    else:
        wWAID = wWAID[10:20]


    #Value di ogni task
    Value = []
    for i in tasks["Value"]:
        Value.append(int(i))

    if (WAID[9] == 10):
        Value = Value[0:10]

    else:
        Value = Value[10:20]


    #Peso del Value (coeffs 1, 4, 7, ..., 88)
    wValue = []

    j = 1
    for k in range(30):
        wValue.append(coeffs.iloc[j].values.tolist())
        j = j + 3

    if (WAID[9] == 10):
        wValue = wValue[0:10]

    else:
        wValue = wValue[10:20]


    #Complessità di ogni task
    Difficulty = []
    for i in tasks["Difficulty"]:
        Difficulty.append(float(i))

    if (WAID[9] == 10):
        Difficulty = Difficulty[0:10]

    else:
        Difficulty = Difficulty[10:20]


    #Peso della Complessità (coeffs 2, 5, 8, ..., 89)
    wDifficulty = []

    j = 2
    for k in range(30):
        wDifficulty.append(coeffs.iloc[j].values.tolist())
        j = j + 3

    if (WAID[9] == 10):
        wDifficulty = wDifficulty[0:10]

    else:
        wDifficulty = wDifficulty[10:20]


    #Stamina consumata da ogni task
    Effort = []
    for i in tasks["Effort Required"]:
        Effort.append(int(i))

    if (WAID[9] == 10):
        Effort = Effort[0:10]

    else:
        Effort = Effort[10:20]

    #Peso della Stamina consumata (coeffs 3, 6, 9, ..., 90)
    wEffort = []

    j = 3
    for k in range(30):
        wEffort.append(coeffs.iloc[j].values.tolist())
        j = j + 3

    if (WAID[9] == 10):
        wEffort = wEffort[0:10]

    else:
        wEffort = wEffort[10:20]

    #Reputazione del WA prima del round
    Rep = list(decisions["Worker Agent Reputation"])


    #Peso della reputation
    wRep = coeffs.iloc[92].values.tolist()

    if (WAID[9] == 10):
        wRep = wRep[0:10]

    else:
        wRep = wRep[10:20]

    #creo i range utili per la creazione di variabili
    (A, T) = (range(len(WAID)), range(len(Value)))


    #VARIABILI

    #x[a, t] vale 1 se la task t è assegnata al WA a, 0 altrimenti
    x = [[m.add_var(var_type=BINARY) for t in T] for a in A]

    #y(a) contiene la reputation del WA a fine round
    y = [m.add_var(var_type=CONTINUOUS) for a in A]


    #VINCOLI

    #Vincolo che impone che ogni task venga assegnata ad uno ed un solo WA
    for t in T:
        m += xsum(x[a][t] for a in A) == 1


    #OBIETTIVO
    #Massimizzare la sommatoria delle reputation dei worker agent
    m.objective = maximize(xsum(dID[a]*wdID[a] for a in A) + xsum(wValue[a][t]*Value[t]*x[a][t] + wDifficulty[a][t]*Difficulty[t]*x[a][t] + wEffort[a][t]*Effort[t]*x[a][t] for t in T for a in A) + xsum(wRep[a]*Rep[a] + wWAID[a]*WAID[a] for a in A))

    m.write("model.lp")
    m.optimize()

    print('Totale MIP: {}'.format(m.objective_value))
    
    decisions2 = pd.read_csv("../Dataset 2.0/Decisions.csv")

    #Da "Decisions.csv" seleziono solo le tuple riguardanti il round successivo
    decisions2 = decisions2[decisions2["Round"] == 10]

    #Estraggo "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
    decisions2 = decisions2.iloc[:, [1, 3, 4]]
    decisions2 = decisions2[decisions2["Session ID"] == i]
    tmp1 = pd.merge(decisions1, decisions2, left_on = "Worker Agent ID", right_on = "Worker Agent ID")

    tmp1 = tmp1.iloc[:, [0, 1, 2, 3, 4, 6]]


    totale = 0
    for k in range(len(tmp1)):
        totale += tmp1.iloc[k, 5]
    print(f"Totale HUMAN: {totale}")



02647ffb-6535-4f65-adc5-bc7e0e5bbc66
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming relaxation problem using Dual Simplex

Coin0506I Presolve 0 (-10) rows, 0 (-110) columns and 0 (-100) elements
Clp0000I Optimal - objective value 0.038234796
Coin0511I After Postsolve, objective 0.038234796, infeasibilities - dual 0.025502339 (8), primal 0 (0)
Coin0512I Presolved model was optimal, full model needs cleaning up
Clp0006I 0  Obj 0.038234796 Dual inf 0.025495139 (8)
Clp0000I Optimal - objective value 0.038234796
Clp0032I Optimal objective 0.03823479575 - 5 iterations time 0.002, Presolve 0.00

Starting MIP optimization
Totale MIP: 2.9151778360195286
Totale HUMAN: 0


04405414-779d-437e-bc69-acf40f360ebc
Cgl0004I proce

Totale HUMAN: 0


1b6a2d87-f9ed-4801-8e51-d9a81dcba0c5
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming relaxation problem using Dual Simplex

Coin0506I Presolve 0 (-10) rows, 0 (-110) columns and 0 (-100) elements
Clp0000I Optimal - objective value 0.038234796
Coin0511I After Postsolve, objective 0.038234796, infeasibilities - dual 0.025502339 (8), primal 0 (0)
Coin0512I Presolved model was optimal, full model needs cleaning up
Clp0006I 0  Obj 0.038234796 Dual inf 0.025495139 (8)
Clp0000I Optimal - objective value 0.038234796
Clp0032I Optimal objective 0.03823479575 - 5 iterations time 0.002, Presolve 0.00

Starting MIP optimization
Totale MIP: 2.9143007693565233
Totale HUMAN: 0


1d9cd410-843d-4b01-8720-d7e4910d91c

IndexError: list index out of range